# Setup

In [1]:
import gc
import time
import warnings
import subprocess

gc.enable()
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb
import optuna

SEED = 55

In [2]:
assert xgb.__version__ == '2.0.2', 'XGBoost version differs from original notebook.' 

In [3]:
#Check GPU availability
try:
    subprocess.check_output('nvidia-smi')
    DEVICE = 'cuda'
except Exception:
    DEVICE = 'cpu'

print(f'Available device: {DEVICE}')

Available device: cuda


In [4]:
DATA_DIR = '/kaggle/input/bitgrit-ai-generated-text-classification/ai-text-competition'
train = pd.read_csv(f'{DATA_DIR}/training_set.csv')
test = pd.read_csv(f'{DATA_DIR}/test_set.csv')
sample_sub = pd.read_csv(f'{DATA_DIR}/solution_format.csv')

# Data overview

In [5]:
train.sample()

,ID,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200,feature_201,feature_202,feature_203,feature_204,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_227,feature_228,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,feature_241,feature_242,feature_243,feature_244,feature_245,feature_246,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,feature_257,feature_258,feature_259,feature_260,feature_261,feature_262,feature_263,feature_264,feature_265,feature_266,feature_267,feature_268,feature_269,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,feature_277,feature_278,feature_279,feature_280,feature_281,feature_282,feature_283,feature_284,feature_285,feature_286,feature_287,feature_288,feature_289,feature_290,feature_291,feature_292,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300,feature_301,feature_302,feature_303,feature_304,feature_305,feature_306,feature_307,feature_308,feature_309,feature_310,feature_311,feature_312,feature_313,feature_314,feature_315,feature_316,feature_317,feature_318,feature_319,feature_320,feature_321,feature_322,feature_323,feature_324,feature_325,feature_326,feature_327,feature_328,feature_329,feature_330,feature_331,feature_332,feature_333,feature_334,feature_335,feature_336,feature_337,feature_338,feature_339,feature_340,feature_341,fe

In [6]:
test.sample()

,ID,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200,feature_201,feature_202,feature_203,feature_204,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_227,feature_228,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,feature_241,feature_242,feature_243,feature_244,feature_245,feature_246,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,feature_257,feature_258,feature_259,feature_260,feature_261,feature_262,feature_263,feature_264,feature_265,feature_266,feature_267,feature_268,feature_269,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,feature_277,feature_278,feature_279,feature_280,feature_281,feature_282,feature_283,feature_284,feature_285,feature_286,feature_287,feature_288,feature_289,feature_290,feature_291,feature_292,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300,feature_301,feature_302,feature_303,feature_304,feature_305,feature_306,feature_307,feature_308,feature_309,feature_310,feature_311,feature_312,feature_313,feature_314,feature_315,feature_316,feature_317,feature_318,feature_319,feature_320,feature_321,feature_322,feature_323,feature_324,feature_325,feature_326,feature_327,feature_328,feature_329,feature_330,feature_331,feature_332,feature_333,feature_334,feature_335,feature_336,feature_337,feature_338,feature_339,feature_340,feature_341,fe

In [7]:
sample_sub.head()

,ID,ind
0,1,0
1,2,0
2,3,0
3,4,1
4,5,1


In [8]:
train.shape, test.shape, sample_sub.shape

((11144, 772), (2786, 771), (2786, 2))

In [9]:
train.isna().sum().sum(), test.isna().sum().sum()

(0, 0)

In [10]:
train = train.drop('ID', axis=1)
test = test.drop('ID', axis=1)

features = ['word_count', 'punc_num'] + [f'feature_{i}' for i in range(768)]
TARGET = 'ind'

In [11]:
train[TARGET].value_counts(normalize=True)

ind
0    0.901292
1    0.098708
Name: proportion, dtype: float64

# Modeling framework

In [12]:
def probs_to_labels(probs, threshold=0.5):
    return (probs >= threshold).astype('int')

In [13]:
def get_best_threshold(y_true, y_probs):
    candidates = np.arange(0, 1, 0.0005)
    scores = [f1_score(y_true, probs_to_labels(y_probs, t)) for t in candidates]
    best_threshold = candidates[np.argmax(scores)]
    return best_threshold

In [14]:
def custom_cv(features, model, folds=7, seed=SEED):
    X, y = train[features], train[TARGET]
    X_test = test[features]
    
    oof_probs = {}
    oof_preds = {}
    test_probs = {}
    test_preds = {}
    scores = []

    cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
    for fold, (train_ids, val_ids) in enumerate(cv.split(X, y)):
        X_train, y_train = X.iloc[train_ids], y.iloc[train_ids]
        X_val, y_val = X.iloc[val_ids], y.iloc[val_ids]

        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False)

        val_probs = model.predict_proba(X_val)[:, 1]
        oof_probs.update(dict(zip(val_ids, val_probs)))
        
        best_threshold = get_best_threshold(y_val, val_probs)
        val_preds = probs_to_labels(val_probs, best_threshold)
        oof_preds.update(dict(zip(val_ids, val_preds)))
        
        test_probs[f'fold{fold}'] = model.predict_proba(X_test)[:, 1]
        test_preds[f'fold{fold}'] = probs_to_labels(test_probs[f'fold{fold}'], best_threshold)

        score = f1_score(y_val, val_preds)
        scores.append(score)
        print(f'Fold #{fold}: {score:.5f} ({model.best_iteration} rounds)')        
        _ = gc.collect()

    test_probs = pd.DataFrame.from_dict(test_probs)
    test_probs['mean'] = test_probs.mean(axis=1) # mean of fold-wise probabilities
    
    test_preds = pd.DataFrame.from_dict(test_preds)
    test_preds['mode'] = test_preds.mode(axis=1)[0].astype('int') # mode of fold-wise predictions
    
    oof_probs = pd.Series(oof_probs).sort_index()
    oof_preds = pd.Series(oof_preds).sort_index()
    print(f'\nAvg. score: {np.mean(scores):.5f} +/- {np.std(scores):.5f}')
    print(f'OOF score: {f1_score(y, oof_preds):.5f}\n')
    
    return oof_probs, oof_preds, test_probs, test_preds

# Experiments

### Expt 1 - eval_metric='aucpr', full features

In [15]:
model1 = xgb.XGBClassifier(
    n_estimators=10000,
    learning_rate=0.01,
    early_stopping_rounds=200,
    objective='binary:logistic',
    eval_metric='aucpr',
    booster='gbtree',
    tree_method='hist',
    device=DEVICE,
    verbosity=0,
    n_jobs=-1,
    random_state=SEED)

In [16]:
%%time
preds1 = custom_cv(features, model1)

Fold #0: 0.62731 (856 rounds)
Fold #1: 0.61864 (3815 rounds)
Fold #2: 0.69403 (6018 rounds)
Fold #3: 0.71014 (2932 rounds)
Fold #4: 0.66421 (3651 rounds)
Fold #5: 0.66667 (2807 rounds)
Fold #6: 0.71942 (5582 rounds)

Avg. score: 0.67149 +/- 0.03610
OOF score: 0.67271

CPU times: user 10min 31s, sys: 1.5 s, total: 10min 32s
Wall time: 10min 14s


In [17]:
threshold1 = get_best_threshold(train[TARGET], preds1[0])
print(f'Best threshold: {threshold1}')
op1 = probs_to_labels(preds1[0], threshold1)
print(f'OOF score: {f1_score(train[TARGET], op1):.5f}\n')
tp1 = probs_to_labels(preds1[2]['mean'], threshold1)

Best threshold: 0.1315
OOF score: 0.65788



### Expt 2 - eval_metric='auc', full features

In [18]:
model2 = xgb.XGBClassifier(
    n_estimators=10000,
    learning_rate=0.01,
    early_stopping_rounds=200,
    objective='binary:logistic',
    eval_metric='auc',
    booster='gbtree',
    tree_method='hist',
    device=DEVICE,
    verbosity=0,
    n_jobs=-1,
    random_state=SEED)

In [19]:
%%time
preds2 = custom_cv(features, model2)

Fold #0: 0.64516 (2024 rounds)
Fold #1: 0.61603 (3500 rounds)
Fold #2: 0.66667 (1857 rounds)
Fold #3: 0.70290 (3001 rounds)
Fold #4: 0.64727 (2097 rounds)
Fold #5: 0.66667 (1840 rounds)
Fold #6: 0.71942 (5750 rounds)

Avg. score: 0.66630 +/- 0.03273
OOF score: 0.66774

CPU times: user 8min 51s, sys: 1.11 s, total: 8min 52s
Wall time: 8min 34s


In [20]:
threshold2 = get_best_threshold(train[TARGET], preds2[0])
print(f'Best threshold: {threshold2}')
op2 = probs_to_labels(preds2[0], threshold2)
print(f'OOF score: {f1_score(train[TARGET], op2):.5f}\n')
tp2 = probs_to_labels(preds2[2]['mean'], threshold2)

Best threshold: 0.1275
OOF score: 0.65650



### Expt 3 - eval_metric='aucpr', only embedding features

In [22]:
features[2], features[-1]

('feature_0', 'feature_767')

In [24]:
%%time
# using same model as Expt 1
preds3 = custom_cv(features[2:], model1)

Fold #0: 0.64567 (3001 rounds)
Fold #1: 0.61475 (1987 rounds)
Fold #2: 0.71429 (4172 rounds)
Fold #3: 0.66929 (1195 rounds)
Fold #4: 0.65414 (3344 rounds)
Fold #5: 0.66202 (2038 rounds)
Fold #6: 0.71062 (8824 rounds)

Avg. score: 0.66725 +/- 0.03277
OOF score: 0.66880

CPU times: user 10min 6s, sys: 1.09 s, total: 10min 8s
Wall time: 9min 49s


In [25]:
threshold3 = get_best_threshold(train[TARGET], preds3[0])
print(f'Best threshold: {threshold3}')
op3 = probs_to_labels(preds3[0], threshold3)
print(f'OOF score: {f1_score(train[TARGET], op3):.5f}\n')
tp3 = probs_to_labels(preds3[2]['mean'], threshold3)

Best threshold: 0.125
OOF score: 0.64916



### Expt 4 - eval_metric='aucpr', only non-embedding features

In [26]:
%%time
# using same model as Expt 1
preds4 = custom_cv(features[:2], model1)

Fold #0: 0.31351 (418 rounds)
Fold #1: 0.32558 (428 rounds)
Fold #2: 0.30380 (135 rounds)
Fold #3: 0.30978 (165 rounds)
Fold #4: 0.33952 (591 rounds)
Fold #5: 0.31325 (134 rounds)
Fold #6: 0.30693 (16 rounds)

Avg. score: 0.31605 +/- 0.01152
OOF score: 0.31508

CPU times: user 42.8 s, sys: 68 ms, total: 42.8 s
Wall time: 41.2 s


In [27]:
threshold4 = get_best_threshold(train[TARGET], preds4[0])
print(f'Best threshold: {threshold4}')
op4 = probs_to_labels(preds4[0], threshold4)
print(f'OOF score: {f1_score(train[TARGET], op4):.5f}\n')
tp4 = probs_to_labels(preds4[2]['mean'], threshold4)

Best threshold: 0.1665
OOF score: 0.30280



### Expt 5 - eval_metric='aucpr' + class balancing, full features

In [34]:
class_ratio = train[TARGET].value_counts()[0] / train[TARGET].value_counts()[1]

In [35]:
model5 = xgb.XGBClassifier(
    scale_pos_weight=class_ratio,
    n_estimators=10000,
    learning_rate=0.01,
    early_stopping_rounds=200,
    objective='binary:logistic',
    eval_metric='aucpr',
    booster='gbtree',
    tree_method='hist',
    device=DEVICE,
    verbosity=0,
    n_jobs=-1,
    random_state=SEED)

In [36]:
%%time
preds5 = custom_cv(features, model5)

Fold #0: 0.64662 (3772 rounds)
Fold #1: 0.62614 (2419 rounds)
Fold #2: 0.72535 (3745 rounds)
Fold #3: 0.68750 (4938 rounds)
Fold #4: 0.67568 (4021 rounds)
Fold #5: 0.64706 (2038 rounds)
Fold #6: 0.71739 (7608 rounds)

Avg. score: 0.67510 +/- 0.03477
OOF score: 0.67384

CPU times: user 12min 5s, sys: 1.27 s, total: 12min 7s
Wall time: 11min 48s


In [39]:
threshold5 = get_best_threshold(train[TARGET], preds5[0])
print(f'Best threshold: {threshold5}')
op5 = probs_to_labels(preds5[0], threshold5)
print(f'OOF score: {f1_score(train[TARGET], op5):.5f}\n')
tp5 = probs_to_labels(preds5[2]['mean'], threshold5)

Best threshold: 0.1685
OOF score: 0.66369



### Expt 6 - eval_metric='auc' + class balancing, full features

In [37]:
model6 = xgb.XGBClassifier(
    scale_pos_weight=class_ratio,
    n_estimators=10000,
    learning_rate=0.01,
    early_stopping_rounds=200,
    objective='binary:logistic',
    eval_metric='auc',
    booster='gbtree',
    tree_method='hist',
    device=DEVICE,
    verbosity=0,
    n_jobs=-1,
    random_state=SEED)

In [38]:
%%time
preds6 = custom_cv(features, model6)

Fold #0: 0.61600 (1113 rounds)
Fold #1: 0.61176 (1021 rounds)
Fold #2: 0.67153 (1716 rounds)
Fold #3: 0.69314 (3734 rounds)
Fold #4: 0.66667 (2222 rounds)
Fold #5: 0.65781 (2317 rounds)
Fold #6: 0.71475 (7499 rounds)

Avg. score: 0.66167 +/- 0.03489
OOF score: 0.66359

CPU times: user 8min 57s, sys: 774 ms, total: 8min 58s
Wall time: 8min 40s


In [40]:
threshold6 = get_best_threshold(train[TARGET], preds6[0])
print(f'Best threshold: {threshold6}')
op6 = probs_to_labels(preds6[0], threshold6)
print(f'OOF score: {f1_score(train[TARGET], op6):.5f}\n')
tp6 = probs_to_labels(preds6[2]['mean'], threshold6)

Best threshold: 0.281
OOF score: 0.64175



### Expt 7 - best config + different folds

In [41]:
%%time
preds7_1 = custom_cv(features, model5 ,folds=5)

Fold #0: 0.63980 (7404 rounds)
Fold #1: 0.67532 (5597 rounds)
Fold #2: 0.65641 (2081 rounds)
Fold #3: 0.67513 (6731 rounds)
Fold #4: 0.66508 (5166 rounds)

Avg. score: 0.66235 +/- 0.01329
OOF score: 0.66230

CPU times: user 10min 14s, sys: 627 ms, total: 10min 15s
Wall time: 10min 2s


In [42]:
%%time
preds7_2 = custom_cv(features, model5 ,folds=10)

Fold #0: 0.66667 (3776 rounds)
Fold #1: 0.63810 (2045 rounds)
Fold #2: 0.65909 (6429 rounds)
Fold #3: 0.72889 (2220 rounds)
Fold #4: 0.64975 (3267 rounds)
Fold #5: 0.69000 (1979 rounds)
Fold #6: 0.67677 (3808 rounds)
Fold #7: 0.66667 (2410 rounds)
Fold #8: 0.64130 (2490 rounds)
Fold #9: 0.71717 (3710 rounds)

Avg. score: 0.67344 +/- 0.02901
OOF score: 0.67441

CPU times: user 14min 40s, sys: 1.09 s, total: 14min 41s
Wall time: 14min 15s


In [43]:
%%time
preds7_3 = custom_cv(features, model5 ,folds=15)

Fold #0: 0.68702 (2078 rounds)
Fold #1: 0.67626 (2846 rounds)
Fold #2: 0.58209 (1147 rounds)
Fold #3: 0.60870 (2525 rounds)
Fold #4: 0.72581 (4169 rounds)
Fold #5: 0.77027 (1963 rounds)
Fold #6: 0.64062 (2587 rounds)
Fold #7: 0.71724 (2474 rounds)
Fold #8: 0.63492 (780 rounds)
Fold #9: 0.68657 (1709 rounds)
Fold #10: 0.65693 (1526 rounds)
Fold #11: 0.67187 (1073 rounds)
Fold #12: 0.65625 (1600 rounds)
Fold #13: 0.72464 (3650 rounds)
Fold #14: 0.71642 (5829 rounds)

Avg. score: 0.67704 +/- 0.04785
OOF score: 0.67873

CPU times: user 17min 46s, sys: 1.47 s, total: 17min 48s
Wall time: 17min 8s


### Expt 8 - best config + best folds + different seeds

In [46]:
%%time
preds8_1 = custom_cv(features, model5, folds=15, seed=2311)

Fold #0: 0.64103 (2098 rounds)
Fold #1: 0.61111 (2147 rounds)
Fold #2: 0.72868 (1197 rounds)
Fold #3: 0.64662 (1165 rounds)
Fold #4: 0.60000 (105 rounds)
Fold #5: 0.68702 (2300 rounds)
Fold #6: 0.75362 (2502 rounds)
Fold #7: 0.64912 (1404 rounds)
Fold #8: 0.66667 (1876 rounds)
Fold #9: 0.68750 (2414 rounds)
Fold #10: 0.72464 (2274 rounds)
Fold #11: 0.75385 (1191 rounds)
Fold #12: 0.70588 (2444 rounds)
Fold #13: 0.64286 (1528 rounds)
Fold #14: 0.63448 (1089 rounds)

Avg. score: 0.67554 +/- 0.04766
OOF score: 0.67414

CPU times: user 14min 16s, sys: 1.32 s, total: 14min 17s
Wall time: 13min 38s


In [47]:
%%time
preds8_2 = custom_cv(features, model5, folds=15, seed=152)

Fold #0: 0.66225 (3487 rounds)
Fold #1: 0.75556 (1301 rounds)
Fold #2: 0.64748 (2142 rounds)
Fold #3: 0.68148 (2342 rounds)
Fold #4: 0.64052 (1586 rounds)
Fold #5: 0.68376 (2344 rounds)
Fold #6: 0.65487 (2274 rounds)
Fold #7: 0.68531 (2522 rounds)
Fold #8: 0.63946 (1934 rounds)
Fold #9: 0.63866 (932 rounds)
Fold #10: 0.64384 (1337 rounds)
Fold #11: 0.68056 (1882 rounds)
Fold #12: 0.66667 (2609 rounds)
Fold #13: 0.65672 (2168 rounds)
Fold #14: 0.73171 (3248 rounds)

Avg. score: 0.67126 +/- 0.03289
OOF score: 0.67062

CPU times: user 16min 41s, sys: 1.49 s, total: 16min 43s
Wall time: 16min 4s


# Submission files

In [44]:
def create_submission_files(preds, config):
    sub = sample_sub.copy()
    sub[TARGET] = preds[-1]['mode']
    sub.to_csv(f'{config}.csv', index=False)

In [45]:
create_submission_files(preds1, 'exp1')
create_submission_files(preds2, 'exp2')
create_submission_files(preds3, 'exp3')
create_submission_files(preds4, 'exp4')
create_submission_files(preds5, 'exp5')
create_submission_files(preds6, 'exp6')
create_submission_files(preds7_1, 'exp7_1')
create_submission_files(preds7_2, 'exp7_2')
create_submission_files(preds7_3, 'exp7_3')

In [48]:
create_submission_files(preds8_1, 'exp8_1')
create_submission_files(preds8_2, 'exp8_2')

**Time to submit!**